# Parsing Practice

The purpose of this practical session is to gain more insight into using parsers and grammars and to get familiarized with the tools provided to this end by NLTK.

**Our goals are to:**

- Understand the process of parsing better by doing some examples by hand.
- Gain insight into grammars by writing a grammar of our own.
- Experiment with training a grammar from a corpus and evaluating it.

**The practical session is organized in four sections:**

- The warm-up, where you are going to use a given toy grammar to generate parse trees and get a chance to compare your intuition with the results provided by the NLTK parsers.
- Then, you will need to design and test your own grammar, again, for a small example.
- Next, you will automatically extract a grammar from a corpus of annotated sentences.
- Finally, you are going to explore ways to improve automatically extracted grammars, through error analysis.

Before diving in, it would be useful to have a look at how NLTK handles parsing. At the end of the Notebook, you will find a summary of the most useful commands. For more helpful explanations and examples, feel free to consult the [Chapter 8 of the NLTK Book](https://www.nltk.org/book/ch08.html), along with the chapter extras.



# Exercise 0

Have a look at the texts and at the grammar below.

### Input texts:

    I saw an elephant
    I saw an elephant in my pajamas

### Grammar:                

    S   -> NP VP     [1.0]
    PP  -> P NP      [1.0]
    NP  -> Det N     [0.4]
    NP  -> Det N PP  [0.2]
    VP  -> V NP      [0.5]
    VP  -> VP PP     [0.5]

    NP  -> 'I'        [0.4]
    Det -> 'an'       [0.5]
    Det -> 'my'       [0.5]
    N   -> 'elephant' [0.5]
    N   -> 'pajamas'  [0.5]
    V   -> 'saw'      [1.0]
    P   -> 'in'       [1.0]



1.- Load the previous grammar by means of the `fromstring` method of the `PCFG` class.

2.- Using this grammar, try to parse the sample sentences with the NLTK `ChartParser` and then with the  NLTK `Viterbi` parser. This parsers covers the two methodologies that we explained in class. The `ChartParser` does not care about probabilities; it just try to find all the possible parses (trees) that fit the sentence. In contrast, Viterbi is a Dynamic Programming algorithm (very similar to the CKY algorithm that we explained in class) that tries to recursively find the best parse (tree) for the sentence, based on the probabilities defined by the grammar.

3.- Examine the output of the parsers. Is there any difference?

4.- Using the NLTK GUI, draw the most probable parse trees found by the NLTK parsers and compare them. 

How are they different?
Do they differ only in the names of the nonterminals use, or is a structural difference as well?
Why do you think these differences occur?



# Solution 0

In [1]:
import nltk
from nltk.tree import *
from nltk.draw import tree
 
# Grammar creation
grammar = nltk.PCFG.fromstring("""
S -> NP VP 	[1.0]
PP -> P NP 	[1.0]
NP -> Det N 	[0.4]
NP -> Det N PP 	[0.2] 
NP -> 'I' 	[0.4]
VP -> V NP 	[0.5]
VP -> VP PP 	[0.5]
Det -> 'an' 	[0.5] 
Det -> 'my' 	[0.5]
N -> 'elephant' [0.5] 
N ->  'pajamas' [0.5]
V -> 'saw' 	[1.0]
P -> 'in' 	[1.0]
""")
 
# Import example sentences to NLTK and tokenize them
str_sentence1 = "I saw an elephant"
str_sentence2 = "I saw an elephant in my pajamas"
 
print("Example sentences")
print(str_sentence1)
print(str_sentence2)
tokens1 = nltk.word_tokenize(str_sentence1)
tokens2 = nltk.word_tokenize(str_sentence2)
 
# Create the Chart and Viterbi parsers, with the input grammar 
chart_parser = nltk.ChartParser(grammar)
viterbi_parser = nltk.ViterbiParser(grammar)
 
# Results for the Chart Parser
print("Parse trees obtained with the Chart parser")
print("Sentence 1")
for tree in chart_parser.parse(tokens1):
    print(tree)
    tree.draw()

print("Sentence 2")
for tree in chart_parser.parse(tokens2):
    print(tree)
    tree.draw() 

# Results for the Viterbi Parser
print("Parse trees obtained with the Viterbi parser")
print("Sentence 1")
for tree in viterbi_parser.parse(tokens1):
    print(tree)
    tree.draw()

print("Sentence 2")
for tree in viterbi_parser.parse(tokens2):
    print(tree)
    tree.draw()


Example sentences
I saw an elephant
I saw an elephant in my pajamas
Parse trees obtained with the Chart parser
Sentence 1
(S (NP I) (VP (V saw) (NP (Det an) (N elephant))))
Sentence 2
(S
  (NP I)
  (VP
    (VP (V saw) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V saw)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))
Parse trees obtained with the Viterbi parser
Sentence 1
(S (NP I) (VP (V saw) (NP (Det an) (N elephant)))) (p=0.02)
Sentence 2
(S
  (NP I)
  (VP
    (V saw)
    (NP
      (Det an)
      (N elephant)
      (PP (P in) (NP (Det my) (N pajamas)))))) (p=0.001)


We can notice that for the first sentence, the two the parsers agree on the result. This is due to the more simple structure of the sentence. For the second sentence, the Chart parser finds two parse trees, while the Viterbi parser outputs only one result. This occurs because the Viterbi parser only computes the most probable parse of the sentences, while the Chart parser is not concerned with the probabilities and it searches for all parse possibilities. In this case, we can notice that the most probable parse for the second sentence is the second parse produced by the Chart parser.


# Exercise 1

You are given  the following sentences:

    Alice wondered with Bob in the empty city streets
    Bob offered Alice an iguana for her birthday
    Alice gave an inspiring speech at the conference on education

1. Design a non-probabilistic context-free grammar for parsing these sentences and then manually provide the parse tree(s) for these sentences. Here is a list of lexical rules and syntactic categories that you could use to devise the grammar:
    Lexical rules:

        V    -> 'wondered' | 'offered' | 'gave'
        NP   -> 'Alice' | 'Bob'
        N    -> 'city' | 'streets' | 'iguana' | 'birthday' | 'speech' | 'conference' | 'education'
        Adj  -> 'empty' | 'inspiring'
        Det  -> 'the' | 'an' | 'her'
        Prep -> 'with' | 'in' | 'for' | 'at' | 'on'
        Suggested syntactic categories (non-terminals):

        S - Sentence 
        NP - Noun Phrase 
        VP - Verb Phrase 
        PNP - Prepositional Noun Phrase
2. With the grammar you designed in point (1.), parse the above sentences using NLTK’s Chart parser, described in Section 1.
3. Display and compare the parse trees obtained in points (1.) and (2.).

# Solution 1

**1. Design a non-probabilistic context-free grammar for parsing these sentences and then manually provide the parse tree(s) for these sentences. Here is a list of lexical rules and syntactic categories that you could use to devise the grammar:**

In [2]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | VP NP PNP | VP PNP | V
PNP -> Prep NP | Prep N
NP -> Det N | Det N PNP | Det NP | Adj N | Adj NP | N N
V -> 'wondered' | 'offered' | 'gave'
NP -> 'Alice' | 'Bob'
N -> 'city' | 'streets' | 'iguana' | 'birthday' | 'speech' | 'conference' | 'education'
Adj -> 'empty' | 'inspiring'
Det -> 'the' | 'an' | 'her'
Prep -> 'with' | 'in' | 'for' | 'at' | 'on'
""")


**2. With the grammar you designed in point (1.), parse the above sentences using NLTK’s Chart parser, described in Section 1.**

In [3]:
str_sentence1 = "Alice wondered with Bob in the empty city streets"
str_sentence2 = "Bob offered Alice an iguana for her birthday"
str_sentence3 = "Alice gave an inspiring speech at the conference on education"
tokens1 = nltk.word_tokenize(str_sentence1)
tokens2 = nltk.word_tokenize(str_sentence2)
tokens3 = nltk.word_tokenize(str_sentence3)

**3. Display and compare the parse trees obtained in points (1.) and (2.).**

In [4]:
chart_parser = nltk.ChartParser(grammar)

print("Sentence 1")
for tree in chart_parser.parse(tokens1):
    print(tree)
    tree.draw()
print

print("Sentence 2")
for tree in chart_parser.parse(tokens2):
    print(tree)
    tree.draw() 
print
    
print("Sentence 3")
for tree in chart_parser.parse(tokens3):
    print(tree)
    tree.draw() 
print

Sentence 1
(S
  (NP Alice)
  (VP
    (VP (VP (V wondered)) (PNP (Prep with) (NP Bob)))
    (PNP
      (Prep in)
      (NP (Det the) (NP (Adj empty) (NP (N city) (N streets)))))))
Sentence 2
(S
  (NP Bob)
  (VP
    (VP (V offered) (NP Alice))
    (NP (Det an) (N iguana))
    (PNP (Prep for) (NP (Det her) (N birthday)))))
Sentence 3
(S
  (NP Alice)
  (VP
    (VP
      (VP (V gave) (NP (Det an) (NP (Adj inspiring) (N speech))))
      (PNP (Prep at) (NP (Det the) (N conference))))
    (PNP (Prep on) (N education))))
(S
  (NP Alice)
  (VP
    (VP
      (VP (V gave))
      (NP (Det an) (NP (Adj inspiring) (N speech)))
      (PNP (Prep at) (NP (Det the) (N conference))))
    (PNP (Prep on) (N education))))
(S
  (NP Alice)
  (VP
    (VP (V gave) (NP (Det an) (NP (Adj inspiring) (N speech))))
    (PNP
      (Prep at)
      (NP (Det the) (N conference) (PNP (Prep on) (N education))))))
(S
  (NP Alice)
  (VP
    (VP (V gave))
    (NP (Det an) (NP (Adj inspiring) (N speech)))
    (PNP
      (Prep 

<function print>

# Exercise 2: Extracting a Grammar from a Corpus

We have designed very simple grammars in the previous exercises. Even for these very small examples, designing grammars are time consuming and does not necessarily have the best results. 

In this exericse, we explore a different approach to creating grammars: **automatic extraction from annotated texts**.

We start from an annotated text: the `Treebank corpus` from the NLTK framework, which includes a small subset (5% fragment) of the `Penn Treebank Corpus`. It contains data from `Wall Street Journal` for 1650 sentences:

    ( (S
        (NP-SBJ
          (NP (NNP Pierre) (NNP Vinken) )
          (, ,)
          (ADJP
            (NP (CD 61) (NNS years) )
            (JJ old) )
          (, ,) )
        (VP (MD will)
          (VP (VB join)
            (NP (DT the) (NN board) )
            (PP-CLR (IN as)
              (NP (DT a) (JJ nonexecutive) (NN director) ))
            (NP-TMP (NNP Nov.) (CD 29) )))
        (. .) ))
        
     
The goal is to automatically extract the CFG out of the treebank and then to evaluate and improve the extracted grammar. 

# Solution 2

**1. First, split the corpus into two parts : one for learning the grammar (training set) and one for evaluating it (test set). For the sake of saving time during this practical session, use 3% of the total number of sentences for the test set. Here is how you could make the split.**

        dataset_size = len(treebank.parsed_sents())

        ## here, we define the split percentage for the training set and the
        ## learning set, in our case ~3% and ~97%

        split_size = int(dataset_size * 0.97)
        learning_set = treebank.parsed_sents()[:split_size]
        test_set = treebank.parsed_sents()[split_size:]

   Note that `test_set`  already contains the parses of the sentences. This is what we are going to compare against, in order to assess the quality of the extracted grammar. In addition, we need the raw (not parsed)  format of the sentences in the test set. These raw sentences will be parsed with the extracted grammar and then compared against the reference, stored in test_set.

        # create a set containing the raw sentences
        sents = treebank.sents()
        raw_test_set = [ [ w for w in sents[i] ] for i in range(split_size, dataset_size) ]
    
**Extract the grammar out of the learning corpus** At this point, it might be useful to have another look at the commands provided in the end of the document. Here is a suggestion on how to do it. Note that we still need the entire lexicon (i.e. rules of the type POS -> word) even though the syntactic rules are extracted from only a portion of the treebank. Assume that you have already separated the learning set from the test set and that the learning set is stored in the  learning_set  variable.

        # This is where we will store all of the productions necessary to
        # construct the PCFG.
        tbank_productions = []

        # For all of the (parsed) sentences in the learning set, extract the
        # productions (i.e. extract the rules).
        for sent in learning_set:
            for production in sent.productions():
                tbank_productions.append(production)

        # Now, we will add the lexical rules for the ENTIRE lexicon.
        for word, tag in treebank.tagged_words():

            # for each tagged word, we create a tree containing that
            # lexical rule, in order to be able to add it to our
            # production set tbank_productions.

            t = Tree.fromstring("("+ tag + " " + word  +")")
            for production in t.productions():
                tbank_productions.append(production)

        # At this point, we have the syntactic rules extracted from the
        # learning set and all of the lexical rules. We are ready to extract
        # the PCFG.
        tbank_grammar = nltk.grammar.induce_pcfg(Nonterminal('S'), tbank_productions)
        
Try to have a look at the grammar. How many rules are there? Imagine you would have to correct it, for instance. You certainly will come to the conclusion that this is not feasible. Indeed, in such a framework (automated extraction) the grammars should not  be modified by hand (maybe not even read), but only the corpus has to be changed/corrected/extended.

Before executing the code you need to install the pre-trained parsing models

In [5]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     /home/acastellanos/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [6]:
import nltk
from nltk.corpus import treebank
from nltk.grammar import Nonterminal
from nltk import Tree
 
print("Splitting the Data Set")
# The size of the learning set has to be chosen so that we avoid
# running out of memory when parsing (caused by a too large learned
# grammar). However, a too small size of the learning set, could lead
# to a grammar that is not useful (i.e. it cannot parse sentences
# other than in the learning set).
 
dataset_size = len(treebank.parsed_sents())
split_size = int(dataset_size*0.97)
 
print("Separating Learning Set")
print("Learning set size: " + str(split_size))
learning_set = treebank.parsed_sents()[:split_size]
 
print("Separating Test Set")
print("Test set size: " + str(dataset_size - split_size))
# This set already contains the parses of the sentences in the test
# set. It is what we are going to compare against, in order to assess
# the quality of the extracted grammar.
test_set = treebank.parsed_sents()[split_size:]
 
print("Set containing the raw sentences, to be parsed with the extracted grammar")
sents = treebank.sents()
raw_test_set = [ [ w for w in sents[i] ] for i in range(split_size, dataset_size) ]
 
print("Extract the syntactic (and part of the lexical) rules from the learning set")
 
tbank_productions = []
for sent in learning_set:
    for production in sent.productions():
        tbank_productions.append(production)
        
print(len(tbank_productions))
print("Extract the rest of the lexical rules from the lexicon")
 
for word, tag in treebank.tagged_words():
    t = Tree.fromstring("("+ tag + " " + word  +")")
    for production in t.productions():
        tbank_productions.append(production)
        
print(len(tbank_productions))
 
print("Creating grammar")
tbank_grammar = nltk.grammar.induce_pcfg(Nonterminal('S'), tbank_productions)
 
# Have a look at the grammar:
print(tbank_grammar)

Splitting the Data Set
Separating Learning Set
Learning set size: 3796
Separating Test Set
Test set size: 118
Set containing the raw sentences, to be parsed with the extracted grammar
Extract the syntactic (and part of the lexical) rules from the learning set
173810
Extract the rest of the lexical rules from the lexicon
274486
Creating grammar
Grammar with 21643 productions (start state = S)
    S -> NP-SBJ VP . [0.161156]
    NP-SBJ -> NP , ADJP , [0.000404913]
    NP -> NNP NNP [0.0312283]
    NNP -> 'Pierre' [0.000107788]
    NNP -> 'Vinken' [0.000215575]
    , -> ',' [0.999793]
    ADJP -> NP JJ [0.0150602]
    NP -> CD NNS [0.0108299]
    CD -> '61' [0.00129032]
    NNS -> 'years' [0.0189964]
    JJ -> 'old' [0.00417573]
    VP -> MD VP [0.0529232]
    MD -> 'will' [0.304419]
    VP -> VB NP PP-CLR NP-TMP [0.000142076]
    VB -> 'join' [0.00158825]
    NP -> DT NN [0.0850731]
    DT -> 'the' [0.494578]
    NN -> 'board' [0.00228222]
    PP-CLR -> IN NP [0.682313]
    IN -> 'as' [0

**2. Evaluate the performance of the grammars on the test set (the 3% of the sentences from the treebank that you isolated in the beginning of the exercise). What proportion of sentences from the test set have been parsed as in the reference (i.e. are correctly parsed) by the extracted grammar?**

In [7]:
# From the created gramar, you create a parser to annotate new sentences
print("Initializing parser")
parser = nltk.ViterbiParser(tbank_grammar) # http://www.nltk.org/_modules/nltk/parse/viterbi.html


# Test the extracted grammar with the Viterbi parser on one sentence.
# The Viterbi parser gives the most probable parse tree
# Test all sentences in the test-set and compare them to the reference parsing
 
parse_success = 0;
# for i in range(0, len(raw_test_set)):
for i in range(0, 3):
    print("==== Parsing sentence " + str(i))
    test_sent = raw_test_set[i]
    # This will raise an exception if the tokens in the test_sentence
    # are not covered by the grammar; should not happen.
    tbank_grammar.check_coverage(test_sent)
    print(test_sent)
    print("[" + str(i) + "] Reference parse:")
    print(test_set[i])
    print("[" + str(i) + "] Parse trees:")
    for tree in parser.parse(test_sent):
        print(tree)
        print(test_sent[i] == tree)
        if test_sent[i] == tree:
            ++parse_success
        print(parse_success)
        
print("Successfully parsed sentences: " + str(parse_success) + " out of " + str(3))
#print("Successfully parsed sentences: " + str(parse_success) + " out of " + str(len(test_set)))

Initializing parser
==== Parsing sentence 0
['Companies', 'listed', '*', 'below', 'reported', 'quarterly', 'profit', 'substantially', 'different', 'from', 'the', 'average', 'of', 'analysts', "'", 'estimates', '.']
[0] Reference parse:
(S
  (NP-SBJ
    (NP (NNS Companies))
    (VP (VBN listed) (NP (-NONE- *)) (ADVP-LOC (IN below))))
  (VP
    (VBD reported)
    (NP
      (NP (JJ quarterly) (NN profit))
      (ADJP
        (RB substantially)
        (JJ different)
        (PP
          (IN from)
          (NP
            (NP (DT the) (NN average))
            (PP
              (IN of)
              (NP (NP (NNS analysts) (POS ')) (NNS estimates))))))))
  (. .))
[0] Parse trees:
(S
  (NP-SBJ (NNS Companies))
  (VP
    (VBN listed)
    (NP
      (NP (-NONE- *))
      (PP
        (IN below)
        (NP (VBN reported) (JJ quarterly) (NN profit))))
    (ADVP-PRP
      (RB substantially)
      (JJ different)
      (PP
        (IN from)
        (NP
          (DT the)
          (ADJP
           

**3. We now want to study the size of the grammar with respect to the size of the learning set. Split the former learning set (i.e. the 95%  of the treebank) into two parts: one kept for actual learning and another part which is not used. Do this for several percentages of the learning set (10%, 50%, 70%, 80%, 90%, 100%), several times for each ratio. H ave a look at the resulting grammars and try to understand their differences.**


### Useful NLTK commands

Grammar creation from a given string:

    g = nltk.CFG.fromstring("""<grammar rules here>""")

Tokenize a sentence:

    tokens = nltk.word_tokenize("<sentence>")
    
Create different types of parsers starting from a grammar:

    chart_parser   = nltk.ChartParser(g)
    viterbi_parser = nltk.ViterbiParser(g)

Parse a tokenized text:

    trees = viterbi_parser.parse(tokens) #for example, for the Viterbi parser

Display the parse trees: 

[ If not done yet, you first need to install this library: 
http://tkinter.unpythonic.net/wiki/How_to_install_Tkinter 
]

The command below will generate a popup window, containing the graphical representation of the tree; while the parse tree drawing window is open, your script will be paused. The script will be resumed when you close the window.

    from nltk.tree import *
    from nltk.draw import tree
    tree.draw()

# Additional Materials: Practical Parsing

NLTK provide us a nice way to create our own grammars, as well as to train them based on an annotated dataset.

Nevertheless, as happens with Tagging, most of the times the pre-train modules are enough to solve most of our situation.

My recommendation is to use either [`spacy`](https://spacy.io/) or [`Stanford CoreNLP`](https://stanfordnlp.github.io/CoreNLP/) instead of NLTK for that. The parsers of NLTK are slow and outdated which made them unsuitable for practical situations.

In the following you have an example of usage of Spacy for dependency parsing, applied to a specific tasks

Firstly, you need to load `spacy` with the desired language, English in this case

In [8]:
import spacy

# Import the English model
nlp = spacy.load('en_core_web_sm')

/home/acastellanos/anaconda3/envs/nlp/bin/python: No module named spacy


Now we can use the spacy object (`nlp`) to perform the desired language annotations to our sentences

In [9]:
sent = "I shot an elephant"
doc=nlp(sent)

The doc object now contains the text plus all the annotations

In [10]:
print("Tokens:", [t.text for t in doc])
print("Lemmas:", [t.lemma_ for t in doc])
print("POS Tagging:", [t.pos_ for t in doc])
print("Parsing:", [t.dep_ for t in doc])

Tokens: ['I', 'shot', 'an', 'elephant']
Lemmas: ['-PRON-', 'shoot', 'an', 'elephant']
POS Tagging: ['PRON', 'VERB', 'DET', 'NOUN']
Parsing: ['nsubj', 'ROOT', 'det', 'dobj']


We could use the dependency parsing annotations to identify different parts of the sentences.

In [11]:
print("Subject:", [tok for tok in doc if (tok.dep_ == "nsubj") ])
print("Verb:", [tok for tok in doc if (tok.dep_ == "ROOT") ])
print("Object:", [tok for tok in doc if ("obj" in tok.dep_ ) ])

Subject: [I]
Verb: [shot]
Object: [elephant]


Let's use it to see what are the objects preferred by Truman in his speech.

In [12]:
import nltk
from nltk.corpus import state_union
from collections import Counter

text = state_union.raw("1946-Truman.txt")

def detect_most_common_objects(corpus_text, n_verbs = 10):
    verbs = []
    for sentence in nltk.sent_tokenize(corpus_text):
        doc = nlp(sentence)
        for tok in doc:
            if ("obj" in tok.dep_):
                verbs.append(tok.lemma_)
    return Counter(verbs).most_common(n_verbs)

detect_most_common_objects(text, 50)

[('year', 188),
 ('dollar', 157),
 ('-PRON-', 98),
 ('program', 80),
 ('war', 77),
 ('which', 67),
 ('expenditure', 50),
 ('time', 45),
 ('government', 41),
 ('States', 36),
 ('Congress', 34),
 ('period', 34),
 ('people', 33),
 ('production', 33),
 ('level', 32),
 ('economy', 31),
 ('policy', 29),
 ('service', 28),
 ('legislation', 28),
 ('Government', 27),
 ('price', 27),
 ('month', 26),
 ('force', 25),
 ('country', 25),
 ('world', 25),
 ('increase', 25),
 ('opportunity', 23),
 ('appropriation', 23),
 ('business', 22),
 ('peace', 22),
 ('purpose', 22),
 ('Administration', 22),
 ('income', 22),
 ('responsibility', 21),
 ('June', 21),
 ('rate', 21),
 ('control', 20),
 ('veteran', 20),
 ('system', 19),
 ('future', 19),
 ('power', 19),
 ('end', 19),
 ('nation', 19),
 ('September', 19),
 ('Act', 19),
 ('worker', 19),
 ('activity', 18),
 ('objective', 18),
 ('development', 18),
 ('addition', 18)]